In [3]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime, timedelta
from pandas import DataFrame, concat, date_range, ExcelWriter, to_datetime, read_csv
import os
from numpy import isnan, inf, mean
import time
import json
from Calculator import Calculator

from utils import getSchema, changedType
from BacktestBaseFunc import *


parent = os.path.dirname(os.path.abspath("__file__"))
output_path = os.path.join(parent, "Output", "HundredBreakOutL")
if not os.path.isdir(output_path):
    os.makedirs(output_path)

In [4]:
bt_list = StockList()

In [36]:
results = {}
files = [x for x in os.listdir(os.path.join(output_path)) if 'Summary' not in x and '20220618' in x]
for f_name in files:
    results[f_name.split("_")[0]] = read_csv(os.path.join(output_path, f_name))
# for ticker_info in bt_list:
#     print(f"============= Backtest {ticker_info['Ticker']}=============")
#     results[ticker_info["Ticker"]] = read_csv(os.path.join(output_path, f'{ticker_info["Ticker"]}_{bt_day_str}.csv'))

# Plot KBar

In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [44]:
def PlotKBar(ticker, df, bt_result, trade_num=0, ma_len=5, ma_len2=10, ma_len3 = 20, predict_start:datetime=None):
#     df = ReadDailyDataLonger(ticker, date)
    show_trade = bt_result.iloc[trade_num]
    trade_end = to_datetime(show_trade.ExitDate)
    trade_start = to_datetime(show_trade.EntryDate)
    end_date = trade_end
    start_date = end_date+timedelta(-120)
    df.Date = to_datetime(df.Date)
    df = df[(df.Date<=end_date) & (df.Date>=start_date)]
    df['MA'] = df.Close.rolling(ma_len).mean()
    df['MA2'] = df.Close.rolling(ma_len2).mean()
    df['MA3'] = df.Close.rolling(ma_len3).mean()
    df['Adj V'] = df.Volume.apply(lambda x: int(float(x)/1000))
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.3, 
                        subplot_titles=(f'OHLC and MA with Trade of {ticker}, Hold between {trade_start.strftime("%Y-%m-%d")} to {trade_end.strftime("%Y-%m-%d")}', 'Volume'), 
                        row_width=[0.2, 0.7])
    
    fig.add_trace(go.Candlestick(x=df.Date,
                    open=df.Open,
                    high=df.High,
                    low=df.Low,
                    close=df.Close,
                    name="CandleStick"), row=1,col=1)
    fig.add_trace(
        go.Scatter(
            x=df.Date,
            y=df.MA,
            name=f"MA{ma_len}"
        ), row=1,col=1)
    fig.add_trace(
        go.Scatter(
            x=df.Date,
            y=df.MA2,
            name=f"MA{ma_len2}"
        ), row=1,col=1)
    fig.add_trace(
        go.Scatter(
            x=df.Date,
            y=df.MA3,
            name=f"MA{ma_len3}"
        ), row=1,col=1)
#     if predict_start:
    fig.add_trace(
    go.Scatter(
        x=[trade_start]*100,
        y=list(np.linspace(df.Low.min(),df.High.max(), 100)),#[],
        name=f"Trade Entry Date {trade_start.strftime('%Y-%m-%d')}",
        opacity=.9,
        line = dict(color='orangered', width=1, dash='dash')
#             color
    ), row=1,col=1)

    fig.add_trace(
    go.Scatter(
        x=[trade_end]*100,
        y=list(np.linspace(df.Low.min(),df.High.max(), 100)),#[],
        name=f"Trade Exit Date {trade_end.strftime('%Y-%m-%d')}",
        opacity=.9,
        line = dict(color='royalblue', width=1, dash='dash')
#             color
    ), row=1,col=1)
    
    fig.add_trace(go.Bar(x=df.Date, y=df['Adj V'], showlegend=False, name = "Volume"), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.Date, y=df['Adj V'].rolling(5).mean(), showlegend=False, name="VMA5"), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.Date, y=df['Adj V'].rolling(20).mean(), showlegend=False, name="VMA20"), row=2, col=1)
    test_df = df.copy(deep=True)
#     test_df.Date = pd.to_datetime(test_df.Date)
    begin_date, end_date = [start_date, end_date]
    test_df = test_df.set_index("Date").reindex(date_range(begin_date, end_date, freq='D'))
    datebreaks = test_df['Close'][test_df['Close'].isnull()].index
#     fig.update_layout(shapes=[
#         dict(
#             type= 'line',
#             yref= 'paper', y0= [df.Low.min()], y1= [df.High.max()],
#             xref= 'x', x0= [predict_start], x1= [predict_start]
#     )])
    fig.update_xaxes(rangebreaks=[dict(values=datebreaks)])
    fig.show()

In [45]:
ticker = '2330'
dt = datetime(2022,6,18)
bt_period = 20
df = StockInterDay(dt+timedelta(-365*bt_period), dt, ticker)

In [46]:
results[ticker].shape

(22, 16)

In [47]:
PlotKBar(ticker, df, results[ticker], trade_num=1, ma_len=5, ma_len2=10, ma_len3 = 20)